In [1]:
from selenium import webdriver
import time
# import pymysql
import random
import re
from lxml import etree
father_class = ['音乐会', '话剧歌剧', '演唱会', '曲苑杂坛', '展览休闲', '舞蹈芭蕾', '体育']
son_class = [
    ['室内乐及古乐', '独奏'],
    ['话剧', '音乐剧'],
    ['livehouse', '流行', '音乐节'],
    ['戏曲', '相声', '魔术'],
    ['展会','特色体验'],
    ['舞蹈'],
    ['球类运动','田径','电竞','篮球']
]

In [2]:
def get_data(url):
    io = open("res.text", 'w')
    driver = webdriver.Chrome(r'chromedriver')
    driver.get(url)
    time.sleep(3)
    for idx, cls in enumerate(father_class):
        driver.find_element_by_xpath("//*[contains(text(), '{}')]".format(cls)).click()
        time.sleep(1)
        for son_cls in son_class[idx]:
            # print(son_cls)
            driver.find_element_by_xpath("//*[contains(text(), '{}')]".format(son_cls)).click()
            time.sleep(1)
            for i in range(1):
                imgs = driver.find_elements_by_xpath("//img[@alt='项目图片']")
                element = driver.find_elements_by_xpath("//div[@class='items']")
                source = driver.page_source
                html = etree.HTML(source)
                item = html.xpath("//img[@alt='项目图片']")
                for idx, each in enumerate(element):            
                    print('###', cls, son_cls, sep=' ', file = io)
                    result = re.split('[\n]', each.text)
                    for res in result:
                        print(res, end='\t', file=io)
                    print('\nhttps://img.alicdn.com/bao/uploaded'+item[idx].get('src'), file=io)
                    imgs[idx].click()
                    window = driver.window_handles
                    driver.switch_to.window(window[1])
                    details = driver.find_element_by_xpath("//div[@id='detail' and @class='list']")
                    print(details.text, file=io)
                    driver.close()
                    time.sleep(0.5)
                    driver.switch_to.window(window[0])

                # driver.find_element_by_xpath("//button[@class='btn-next']").click()
                # time.sleep(3)
    driver.close()
    io.close()

In [3]:
url = 'https://search.damai.cn/search.html'
# get_data(url)

In [8]:
i = 0
cls_dict = {}
for idx, f_cls in enumerate(father_class):
    cls_dict[f_cls] = i
    i += 1
    for s_cls in son_class[idx]:
        cls_dict[s_cls] = i
        i += 1

In [16]:
f = open('res.text', 'r')
# poster = open('poster.csv', 'w')
# ticket = open('ticket.csv', 'w')
# print('id,name,type,left,price,detail,city,venues,beginTime,endTime', file=ticket)
ticket = open('data.sql', 'w')
print('USE dbTicket;', file=ticket)
print("DELETE FROM ticket_type WHERE TRUE;", file=ticket)
print("DELETE FROM ticket WHERE TRUE;", file=ticket)

for idx, f_cls in enumerate(father_class):
    print('INSERT INTO ticket_type VALUES', file=ticket)
    print("('{}', NULL),".format(f_cls), file=ticket)
    for i in range(len(son_class[idx]) - 1):
        print("('{}', '{}'),".format(son_class[idx][i], f_cls), file=ticket)
    print("('{}', '{}');\n".format(son_class[idx][-1], f_cls), file=ticket)


In [17]:

i = 0
tid = 0
line = ''
line = f.readline().strip()
while i < 300:
    i += 1
# while True:
    line = line.strip()
    tp = line.rsplit(' ', 1)[1]
    sline = f.readline().strip().split('\t')
    # lens = len(sline)
    # print(sline)
    nm = sline[1]
    tname = nm[nm.find('】') + 2:]
    # print(len(tname), end = ',')
    idx = 2
    if sline[idx].startswith('艺人'):
        idx += 1
    elif sline[idx].find('|') == -1:
        while True:
            line = f.readline()
            if not line or line.startswith('###'):
                break
        continue

    hotness, left = random.randint(10, 1000), random.randint(50, 400) 
    # print(hotness)
    # print(left)
    place = sline[idx]
    city, venues = place.split('|')
    city = city.strip()
    venues = venues.strip()
    # print(city)
    # print(venues)
    idx += 1
    
    # date and time
    date_time = sline[idx]
    bgtime = date_time[:10].replace('.', '-')
    # print(bgtime)
    edtime = ''
    if '-' not in date_time:
        if ':' in date_time:
            bgtime += ' ' + date_time[-5:]
        else:
            bgtime += ' 08:00'
        edtime = bgtime
    else:
        if len(date_time) == 21:
            edtime = date_time[-10:].replace('.', '-')
        else:
            edtime = date_time[:4] + date_time[-6:].replace('.', '-')
        bgtime += ' 08:00'
        edtime += ' 08:00'
    # print(date_time)
    # print(bgtime)
    
    line = f.readline().strip()
    url = line
    idx += 1

    if len(sline[idx]) == 3:
        idx += 1
    price = sline[idx][:sline[idx].find('元')-1]
    hprice = price.split('-')[0]
    lprice = price.split('-')[-1]
    
    # print(hprice, lprice)

    # print(line)
    detail = ''
    while True:
        line = f.readline()
        # print(line)
        if not line or line.startswith('###'):
            tid += 1
            print("INSERT INTO ticket VALUE({}, {}, '{}', '{}', '{}', '{}', {}, '{}', '{}', '{}', '{}');".format(tid, left, bgtime, city, detail.replace("'", r"\'")[:3000], edtime, lprice, tname, tp, venues, url), file=ticket)
            break
        line = line.replace('\n', r'\n')
        detail += line
    if not line:
        break

f.close()
# poster.close()
ticket.close()
    

In [14]:
len("2020.12.23-2021.01.03")

21

In [10]:
a = 'a\n'

1